<a href="https://colab.research.google.com/github/OrangeLyx/507final_369/blob/main/code/train_part2_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
!pip install datasets
from datasets import load_dataset,Dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
imdb = load_dataset("imdb")

train_dataset=imdb["train"].shuffle(seed=123)
test_dataset=imdb["test"].shuffle(seed=123)

data_train= pd.DataFrame({"text": train_dataset["text"], "label":train_dataset["label"]})
data_test= pd.DataFrame({"text": test_dataset["text"], "label":test_dataset["label"]})

In [ ]:
def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub(r'<.*?>','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)

    stop_words = {'a', 'an', 'the'}
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

cleaned=lambda x:clean_text(x)

data_test['text']=pd.DataFrame(data_test.text.apply(cleaned))
data_train['text']=pd.DataFrame(data_train.text.apply(cleaned))


data_test.head(10)
data_train.head(10)

,text,label
0,i wont claim to be fan of ralph bakshi because...,1
1,i was not impressed about this film especially...,0
2,aka demons iiimade for italian tv although sho...,0
3,thief of baghdad is one of my ten alltime favo...,1
4,this is first 10 out of 10 that ive given any ...,1
5,there is uk edition to this show which is rath...,0
6,spoilersi saw original on tv sometime ago and ...,1
7,spoilers powerhouse movie that shows how men i...,1
8,this movie is intelligent that is more than mo...,1
9,despite unfortunately thinking itself to be in...,0


In [ ]:
data_all = pd.concat([data_train, data_test], axis=0).reset_index(drop=True)

data_train, data_test= train_test_split(data_all, test_size=0.25, random_state=42, stratify=data_all["label"])

print("Size of train dataset:", len(data_train))
print("Size of test dataset:", len(data_test))
print(data_train["label"].value_counts(normalize=True))
print(data_test["label"].value_counts(normalize=True))

dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)

训练集大小: 37500
验证集大小: 12500
label
0    0.5
1    0.5
Name: proportion, dtype: float64
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
import torch
!pip install evaluate
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, TrainingArguments, Trainer, XLNetModel

tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
# model = XLNetModel.from_pretrained('xlnet-large-cased', num_labels=2)
model.to(device)

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

encoded_train = dataset_train.map(preprocess_function, batched=True)
encoded_test = dataset_test.map(preprocess_function, batched=True)

encoded_train = encoded_train.rename_columns({"label": "labels"})
encoded_test = encoded_test.rename_columns({"label": "labels"})
encoded_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
encoded_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    acc = accuracy_score(labels, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./xlnet_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,        # Adjust based on GPU
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=1e-5,
    weight_decay=0.02,
    logging_dir="./xlnet_logs",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model.resize_token_embeddings(len(tokenizer))
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-28-7d66c1a6d4fd>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.176000,0.196571,0.944640,0.939159,0.950880,0.944983
2,0.152100,0.187793,0.951120,0.941167,0.962400,0.951665
3,0.084600,0.234213,0.950800,0.942794,0.959840,0.951241


TrainOutput(global_step=7032, training_loss=0.15371075526846148, metrics={'train_runtime': 2016.9051, 'train_samples_per_second': 55.779, 'train_steps_per_second': 3.487, 'total_flos': 3.20490314496e+16, 'train_loss': 0.15371075526846148, 'epoch': 3.0})

In [ ]:
trainer.save_model("./xlnet_imdb_model")
tokenizer.save_pretrained("./xlnet_imdb_model")

AttributeError: 'Trainer' object has no attribute 'save_pretrained'

In [ ]:
test_results = trainer.evaluate(encoded_test)
print("Test results:", test_results)

Test results: {'eval_loss': 0.18779315054416656, 'eval_accuracy': 0.95112, 'eval_precision': 0.9411672664684713, 'eval_recall': 0.9624, 'eval_f1': 0.9516652163594652, 'eval_runtime': 97.5489, 'eval_samples_per_second': 128.141, 'eval_steps_per_second': 8.016, 'epoch': 3.0}


In [ ]:
trainer.evaluate()

{'eval_loss': 0.18779315054416656,
 'eval_accuracy': 0.95112,
 'eval_precision': 0.9411672664684713,
 'eval_recall': 0.9624,
 'eval_f1': 0.9516652163594652,
 'eval_runtime': 97.5677,
 'eval_samples_per_second': 128.116,
 'eval_steps_per_second': 8.015,
 'epoch': 3.0}

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("./xlnet_imdb_model")
tokenizer = AutoTokenizer.from_pretrained("./xlnet_imdb_model")